<a href="https://colab.research.google.com/github/MLGlobalHealth/StatML4PopHealth/blob/main/practicals/day1/practical1/1_stan_intro_lab_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/imperial.png" width="250" vspace="8"/>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/mlgh.png" width="220" hspace="50" vspace="5"/>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/ammi.png" width="190"/>

<font size="6">Modern Statistics and Machine Learning
for Population Health in Africa </font>

<font size="4">24th - 28th March 2025</font>

</center>

# Computing lab: Stan Basics
## Modern Statistics and Machine Learning for Population Health in Africa
#### Tristan Naidoo and Sahoko Ishida
(With thanks to Yu Chen and Oliver Ratmann for the content)

## Study objectives

Hello all!

By the end of this lab,

- you will understand the key benefits of modern statistical computing languages such as `Stan`; and
- you will have installed the `Stan` software within `Python`, via `cmdstan`; and
- you will be familiar with a typical analysis workflow using `Stan`, and associated `Python` commands.

## Why Stan?

`Stan`

- is an open-source statistical inference software, which implements gradient-based MCMC to sample from posterior distributions.
- allows us to focus on statistical modelling, rather than implementing inference algorithms.
- algorithms are implemented in \textsl{C++}, and can be accessed through interfaces like `CmdStanPy` and `PyStan` in `Python`.
- Bayesian models are written in text files.

There are several modern alternatives to `Stan`, including

- [numpyro](https://num.pyro.ai/en/latest/index.html#introductory-tutorials); and
- [TMD](https://github.com/kaskr/adcomp/wiki); and
- [Nimble](https://r-nimble.org).

## Installation
For this course we will use `CmdStanPy`, which is a recently developed lightweight alternative to `PyStan` for interfacing with Stan in Python. The functions to compile, run, and access the output of Bayesian inference algorithms are very similar, but not identical to those in the `PyStan` package. However, both `CmdStanPy` and `PyStan` read exactly the same `Stan` model files, and so `CmdStanPy` is an excellent alternative if you run into installation issues with `PyStan`, or if you would rather work with the most recent `Python` interface to `Stan`'s algorithms.

Throughout this course, we will work in Google Colab. The steps to install `CmdStanPy` and `CmdStan` in your Colab instance are provided below.

If you would like to install `CmdStanPy` and `CmdStan` on your local machine please follow the instructions provided [here](https://mc-stan.org/cmdstanpy/installation.html).

#### Step 1: Install the python package `CmdStanPy`

In [ ]:
!pip install --upgrade cmdstanpy==1.2.5

#### Step 2: Install `CmdStan`

We install the pre-built colab CmdStan binary because its faster than compiling from source via `install_cmdstan()` function. If you are installing locally please download the appropriate binary or alternatively use `install_cmdstan()`.

In [ ]:
import os
import shutil
import urllib

In [ ]:
tgz_file = 'colab-cmdstan-2.36.0.tar.gz'
tgz_url = 'https://github.com/stan-dev/cmdstan/releases/download/v2.36.0/colab-cmdstan-2.36.0.tgz'
if not os.path.exists(tgz_file):
    urllib.request.urlretrieve(tgz_url, tgz_file)
    shutil.unpack_archive(tgz_file)

We also need to install an additional linux dependency that is missing on the machine that our colab instance is running on.

In [ ]:
!apt install libtbb2

#### Step 3: Specify `CmdStan` path
We set the `CmdStan` path using an environment variable. This tells your machine where to look for `CmdStan`.

In [ ]:
os.environ['CMDSTAN'] = './cmdstan-2.36.0'

Next, we check `CmdStan` path. This should be the same as what is specified above.

In [ ]:
from cmdstanpy import CmdStanModel, cmdstan_path
cmdstan_path()

## Hello world example, using `CmdStanPy`

Let us start with a very simple example to become familiar with `Stan`.

We aim to fit a normal model to 100 data points $y_i, i=1,\dotsc,n=100$, and estimate the joint posterior distribution of the mean and standard deviation numerically with Stan,

\begin{align*}
&y_{i} \sim \text{Normal}(\mu, \sigma^2) \\
& \mu \sim \text{Normal}(0,100)\\
& \sigma\sim \text{Half-Cauchy}(0,1).
\end{align*}

Let us compile the corresponding `Stan` model and run two Hamiltonian Monte Carlo (HMC) chains for 4,000 iterations each, including 1,000 warmup iterations.

Every `Stan` model files is structured in terms of the following blocks:

- `data` block: specifies data required to fit the model
- `transformed data` block: specifies temporary transformations of the data, e.g. QR decomposition of $X$; variables that do not change
- `parameters` block: specifies all parameters that are fitted; cannot be assigned values directly
- `transformed parameters` block: optional transformations of parameters, e.g. risk differences
- `model` block: specifies the model in terms of likelihood and priors
- `generated quantities` block: quantities that depend on parameters and data, and do not affect inference

Here is a simple implementation of our Hello World normal model:

In [ ]:
# Hello World normal model
model1_text = """
data{
    int<lower=1> N;
    array [N] real y;
}
parameters{
    real mu;
    real<lower=0> sigma;
}
model{
    sigma ~ cauchy( 0 , 1 );
    mu ~ normal( 0 , 10 );
    y ~ normal( mu , sigma );
}
"""

To use this model in `Python`, we start by loading up our `Python` packages.
If you are running this notebook on Google colab the packages we need are included by default. If you are running this notebook locally please install any missing packages.

In [ ]:
import os
import pickle
from pathlib import Path

import arviz as az
from cmdstanpy import CmdStanModel
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme(style="whitegrid")

import logging

logging.getLogger("cmdstanpy").setLevel(logging.INFO)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Adjust this as required - this is where your output will be stored.
output_dir = Path(*["drive", "MyDrive", "StatML4PopHealth", "output"])
output_dir.mkdir(parents=True, exist_ok=True)

Now, please complete the following code block to sample from the joint posterior associated with the Hello World model for 4,000 iterations including 1,000 iterations of warmup. Setup two Hamiltonian Monte Carlo chains and initialise the two chains at $(\mu=1, \sigma=2)$ and $(\mu=-1, \sigma=0.5)$:


In [ ]:
# Set seed for reproducibility
np.random.seed(10680)  # Use your birth date

# Generate random normal data
y = np.random.normal(loc=0, scale=1, size=100)

# Create data dictionary for Stan
stan_data = {
    'N': len(y),
    'y': y
}

# Write the Stan model to a file
# CmdStanPy requires the model to be written to a file
model1_filename = output_dir.joinpath("helloworld.stan")
with open(model1_filename, "w") as f:
    f.write(model1_text)

In [ ]:
# Compile the Stan model using CmdStanPy
model1_compiled = CmdStanModel(stan_file=model1_filename)

In [ ]:
# Sample from the joint posterior of the Hello World model using CmdStanPy
model1_fit = model1_compiled.sample(
    data=stan_data,
    seed=123,
    chains=2,
    parallel_chains=2,
    iter_warmup=1000,
    iter_sampling=4000,
    refresh=500,  # Print update every 500 iterations
    save_warmup=True,
    inits=[{'mu': 1, 'sigma': 2},
           {'mu': -1, 'sigma': 0.5}]  # Initial values for the chains
)

# This won't be included in the chunks below, it's just an example
# in case you want to save your output object
# Save the output to a pickle file
output_fit_file = output_dir.joinpath("model1_fit_cmdstanpy.pkl")
with open(output_fit_file, "wb") as f:
    pickle.dump(model1_fit, f)

Next, we explore the following code snippets that provide key functions to assess the Monte Carlo output.

In [ ]:
# This won't be included in the chunks below, it's just an example
# in case you want to save your output object
# Load the output from the pickle file saved earlier
with open(output_fit_file, "rb") as f:
    model1_fit = pickle.load(f)

# Specify the parameters you want to check
model1_pars = ["mu", "sigma"]
model1_pars_with_lp = model1_pars + ["lp__"]

# Get summary of the specified parameters
model1_summary = model1_fit.summary(sig_figs=4)
model1_summary.loc[model1_pars_with_lp]

In [ ]:
# We can also use a library to get this summary
# Convert the CmdStanPy model fit to ArviZ InferenceData
model1_fit_az_idata = az.from_cmdstanpy(model1_fit, save_warmup=True)

# arviz parses our data into groups based on our fit object
model1_fit_az_idata

In [ ]:
# Get summary statistics, including 2.75% and 97.5% quantiles
summary_stats = az.summary(model1_fit_az_idata,
                           var_names=model1_pars,
                           group="posterior",
                           hdi_prob=0.95,
                           kind="all"# 95% highest density interval
                           )

summary_stats

In [ ]:
# To plot traces and assess convergence, extract
# Monte Carlo samples including warmup in default
# array format that keeps all chains separate

# It's often helpful to plot the log posterior density too. Since it is a
# sample stat and arviz can only access one group at a time, we will manually
# add it to make it easier to view everything together
model1_fit_az_idata["posterior"]["lp"] = model1_fit_az_idata["sample_stats"]["lp"]
model1_pars_with_lp = model1_pars + ["lp"]

# Print summary of the parameters
summary_stats = az.summary(model1_fit_az_idata,
                           var_names=model1_pars_with_lp,
                           group="posterior",
                           hdi_prob=0.95,
                           kind="all"
                          )

summary_stats

In [ ]:
# Plot trace for parameters (mu, sigma, and lp)
# different colours are different chains
az.plot_trace(model1_fit_az_idata,
              filter_vars="like",
              var_names=model1_pars_with_lp,
              combined=False,
              compact=False,
              kind="trace",
              figsize=(11, 10))

plt.show()

# Save the plot to a PDF file
plt.savefig(output_dir.joinpath("model1_cmdstanpy_trace_arviz.pdf"),
            format="pdf",
            bbox_inches="tight")

plt.close()

In [ ]:
# Extract the posterior samples excluding warmup
# Make the pairs plot (including log-posterior)
az.plot_pair(model1_fit_az_idata,
             var_names=model1_pars_with_lp,
             marginals=True,
             figsize=(13, 8))

plt.show()

plt.savefig(output_dir.joinpath("model1_cmdstanpy_pairsplot.pdf"),
            format="pdf", bbox_inches="tight")
plt.close()

In [ ]:
# To manipulate the posterior Monte Carlo samples after warmup we
# typically prefer to extract the samples as a DataFrame (excluding warmup)
posterior_df = model1_fit_az_idata.posterior.to_dataframe()
posterior_df.head()

## The flat priors example

Let us adapt the Hello World example a wee bit, so that we can explore how a non-successful numerical inference run with `Stan` might look like.

We will engineer a statistical problem that is very hard to sample from, due to unsuitable prior specifications. First, we will reduce the data to just two data points. Second, we will specify unbounded uniform priors are specified for both $\mu$ and $\sigma$, which says that prior values close to infinity are as likely as close to zero.

The learning outcome here is that a common reason why `Stan` fails to sample from a joint posterior is that the priors are misspecified as either way too wide, or way too narrow.

In [ ]:
# Data now consist of TWO data points ONLY
y = np.array([-1, 1])
stan_data = {
    'N': len(y),
    'y': y
}

In [ ]:
# Define the Stan model with flat priors on mu and sigma
model2_text = """
data {
    int<lower=1> N;
    array [N] real y;
}
parameters {
    real mu;
    real<lower=0> sigma;
}
model {
    y ~ normal(mu, sigma);
}
"""

Now, let us compile the `Stan` model file, and run `Stan`'s Hamiltonian Monte Carlo algorithm for $1000$ iterations that include $400$ warmup iterations. Setup two Hamiltonian Monte Carlo chains and initialise both chains at $(\mu=0, \sigma=1)$. Then make a trace plot:

In [ ]:
# Write the Stan model to a file
model2_filename = "model2.stan"
with open(model2_filename, 'w') as f:
    f.write(model2_text)

# Compile the Stan model using CmdStanPy
model2_compiled = CmdStanModel(stan_file=model2_filename)

# Sample from the posterior distribution
model2_fit = model2_compiled.sample(
    data=stan_data,
    seed=123,
    chains=2,
    parallel_chains=2,
    iter_sampling=1400,
    iter_warmup=400,
    refresh=500,
    save_warmup=True,
    inits=[{'mu': 0, 'sigma': 1},
          {'mu': 0, 'sigma': 1}]
    )

In [ ]:
# Create an ArviZ InferenceData object
model2_fit_az_idata = az.from_cmdstanpy(model2_fit, save_warmup=True)

# Print the summary of the sampling results
model2_pars = ["mu", "sigma"]

summary_stats = az.summary(model2_fit_az_idata,
                           var_names=model2_pars,
                           group="posterior",
                           hdi_prob=0.95,
                           kind="all")

summary_stats

In [ ]:
# Trace plots
az.plot_trace(model2_fit_az_idata,
              filter_vars="like",
              var_names=["mu", "sigma"],
              combined=False,
              compact=False,
              kind="trace",
              figsize=(11, 10))

plt.show()

# Save the plot to a PDF file
plt.savefig(output_dir.joinpath("model2_cmdstanpy_trace_arviz.pdf"),
            format="pdf",
            bbox_inches="tight")

plt.close()

For comparison, let us now place weakly informative priors on $\mu$ and $\sigma$, and repeat the excercise:

In [ ]:
# Specify Stan model with weakly informative prior on mu and sigma
model3_text = """
data{
    int<lower=1> N;
    array [N] real y;
}
parameters{
    real mu;
    real<lower=0> sigma;
}
model{
    sigma ~ cauchy( 0 , 1 );
    mu ~ normal( 0 , 10 );
    y ~ normal( mu , sigma );
}
"""

In [ ]:
# Write the Stan model to a file
model3_filename = "model3.stan"
with open(model3_filename, 'w') as f:
    f.write(model3_text)

# Compile the Stan model using CmdStanPy
model3_compiled = CmdStanModel(stan_file=model3_filename)

# Sample from the posterior distribution
model3_fit = model3_compiled.sample(
    data=stan_data,
    seed=123,
    chains=2,
    parallel_chains=2,
    iter_sampling=1400,
    iter_warmup=400,
    refresh=500,
    save_warmup=True,
    inits=[{'mu': 0, 'sigma': 1},
          {'mu': 0, 'sigma': 1}]
    )

In [ ]:
# Create an ArviZ InferenceData object

model3_fit_az_idata = az.from_cmdstanpy(model3_fit, save_warmup=True)

# Print the summary of the sampling results

model3_pars = ["mu", "sigma"]

summary_stats = az.summary(model3_fit_az_idata,
                           var_names=model3_pars,
                           group="posterior",
                           hdi_prob=0.95,
                           kind="all")

summary_stats

In [ ]:
# Trace plots
az.plot_trace(model3_fit_az_idata,
              filter_vars="like",
              var_names=["mu", "sigma"],
              combined=False,
              compact=False,
              kind="trace",
              figsize=(11, 10))

plt.show()

# Save the plot to a PDF file
plt.savefig(output_dir.joinpath("model3_cmdstanpy_trace_arviz.pdf"),
            format="pdf",
            bbox_inches="tight")

plt.close()

## The unidentifiable parameters example

Finally, we will adapt the Hello World example slightly differently, again to explore how a non-successful numerical inference run with `Stan` might look like in the case when two parameters cannot be estimated from the data, only their combination. To engineer this situation, we will introduce two unknown random variables $\alpha_1$ and $\alpha_2$, and say that the sum of both is the mean in our Hello World normal model.

The learning outcome here is that a common reason why `Stan` fails to sample from a joint posterior is that the parameters in a model are highly correlated, and there are multiple equally likely solutions under the model as specified.

Let us start with unbounded uniform priors specified on $\alpha_1$ and $\alpha_2$.

In [ ]:
# specify Stan model with unidentifiable parameters and flat prior
model4_text = """
data{
    int<lower=1> N;
    array [N] real y;
}
parameters{
    real alpha1;
    real alpha2;
    real<lower=0> sigma;
}
transformed parameters{
    real mu= alpha1 + alpha2;
}
model{
    sigma ~ cauchy( 0 , 1 );
    y ~ normal( mu , sigma );
}
"""

Let us run `Stan`'s Hamiltonian Monte Carlo algorithm for $1000$ iterations that include $400$ warmup iterations. Setup two Hamiltonian Monte Carlo chains and initialise both chains at $(\mu=0, \sigma=1)$. Then make a trace plot:

In [ ]:
# Write the Stan model to a file
model4_filename = "model4.stan"
with open(model4_filename, 'w') as f:
    f.write(model4_text)

# Compile the Stan model using CmdStanPy
model4_compiled = CmdStanModel(stan_file=model4_filename)

# Sample from the posterior distribution
model4_fit = model4_compiled.sample(
    data=stan_data,
    seed=123,
    chains=2,
    parallel_chains=2,
    iter_sampling=1400,
    iter_warmup=400,
    refresh=500,
    save_warmup=True,
    inits=[{'alpha1': 0, "alpha2":0, 'sigma': 1},
           {'alpha1': 0, "alpha2":0, 'sigma': 1}]
    )

In [ ]:
# Create an ArviZ InferenceData object
model4_fit_az_idata = az.from_cmdstanpy(model4_fit, save_warmup=True)

# Print the summary of the sampling results
model4_pars = ["alpha1", "alpha2", "mu", "sigma"]

summary_stats = az.summary(model4_fit_az_idata,
                           var_names=model4_pars,
                           group="posterior",
                           hdi_prob=0.95,
                           kind="all")

summary_stats

In [ ]:
# Trace plots
az.plot_trace(model4_fit_az_idata,
              filter_vars="like",
              var_names=model4_pars,
              combined=False,
              compact=False,
              kind="trace",
              figsize=(11, 10))

plt.show()

# Save the plot to a PDF file
plt.savefig(output_dir.joinpath("model4_cmdstanpy_trace_arviz.pdf"),
            format="pdf",
            bbox_inches="tight")

plt.close()

In [ ]:
# Plot pairs
az.plot_pair(model4_fit_az_idata,
             var_names=model4_pars,
             marginals=True,
             figsize=(13, 8))

# Save the plot to a PDF file
plt.show()

plt.savefig(output_dir.joinpath("model4_cmdstanpy_pairsplot.pdf"),
            format="pdf", bbox_inches="tight")
plt.close()

For comparison, let us now consider a model with weakly informative $\text{Normal}(0,10^2)$ priors on $\alpha_1$ and $\alpha_2$. Again, let us run `Stan`'s Hamiltonian Monte Carlo algorithm for $1000$ iterations that include $400$ warmup iterations. Setup two Hamiltonian Monte Carlo chains and initialise both chains at $(\alpha_1=0, \alpha_2=0, \sigma=1)$. Then make a trace plot, and also a pair plot as shown above:

In [ ]:
# specify Stan model with unidentifiable parameters and flat prior
model5_text = """
data{
    int<lower=1> N;
    array [N] real y;
}
parameters{
    real alpha1;
    real alpha2;
    real<lower=0> sigma;
}
transformed parameters{
    real mu= alpha1 + alpha2;
}
model{
    sigma ~ cauchy( 0 , 1 );
    alpha1 ~ normal(0, 10);
    alpha2 ~ normal(0, 10);
    y ~ normal( mu , sigma );
}
"""

In [ ]:
# Write the Stan model to a file
model5_filename = "model5.stan"
with open(model5_filename, 'w') as f:
    f.write(model5_text)

# Compile the Stan model using CmdStanPy
model5_compiled = CmdStanModel(stan_file=model5_filename)

# Sample from the posterior distribution
model5_fit = model5_compiled.sample(
    data=stan_data,
    seed=123,
    chains=2,
    parallel_chains=2,
    iter_sampling=4000,
    iter_warmup=1000,
    refresh=500,
    save_warmup=True,
    inits=[{'alpha1': 0, "alpha2":0, 'sigma': 1},
           {'alpha1': 0, "alpha2":0, 'sigma': 1}],
    )

In [ ]:
# Create an ArviZ InferenceData object
model5_fit_az_idata = az.from_cmdstanpy(model5_fit, save_warmup=True)

# Print the summary of the sampling results
model5_pars =  ["alpha1", "alpha2", "mu", "sigma"]

summary_stats = az.summary(model5_fit_az_idata,
                           var_names=model5_pars,
                           group="posterior",
                           hdi_prob=0.95,
                           kind="all")

summary_stats

In [ ]:
# Trace plots
az.plot_trace(model5_fit_az_idata,
              filter_vars="like",
              var_names=model5_pars,
              combined=False,
              compact=False,
              kind="trace",
              figsize=(11, 10))

plt.show()

# Save the plot to a PDF file
plt.savefig(output_dir.joinpath("model5_cmdstanpy_trace_arviz.pdf"),
            format="pdf",
            bbox_inches="tight")

plt.close()

In [ ]:
# Plot pairs
az.plot_pair(model5_fit_az_idata,
             var_names=model5_pars,
             marginals=True,
             figsize=(13, 8))
# Save the plot to a PDF file
plt.show()

plt.savefig(output_dir.joinpath("model5_cmdstanpy_pairsplot.pdf"),
            format="pdf", bbox_inches="tight")
plt.close()


Let us reflect on the trace plots and the pair plots for the different models:

- What do you observe in terms of convergence, mixing, and the HMC traces in general?
- What does the pair plot reveal to you?

The first observation is that provided that the prior distributions on model parameters are suitably specified through e.g. weakly informative priors, then Stan's Hamiltonian Monte Carlo algorithm usually works very well to provide us with Monte Carlo samples from the joint posterior distribution.

The second observation is related to the pairs plot. We can see that several parameters are extremely highly correlated. In fact, the posterior alpha1 is essentially equal to -alpha2. This means that the joint posterior distribution is essentially concentrated on a hyperplane, a set of measure zero. Yet, Stan's algorithm understands the geometry of the posterior distribution, and can efficiently sample from it. This would be very difficult with a standard Metropolis Hastings algorithm.

## Hello world example using `pystan`

Finally, we revisit our Hello World example to see how exactly each step above can be implemented with `pystan`:

In [ ]:
!pip install pystan nest-asyncio

Running in `pystan` in a notebook is problematic since Jupyter blocks certain asyncio functions. To address this problem, we use `nest-asyncio`.

See this [`pystan` FAQ](https://pystan.readthedocs.io/en/latest/faq.html) for more information.

In [ ]:
import stan
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# Make data
np.random.seed(10680)  # Use your birth date
y = np.random.normal(0, 1, 100)  # Generate random data

stan_data = {
    'N': len(y),
    'y': y
}

In [ ]:
# Stan model code (using the same 'model1.stan' as in your R code)
pystan_model1_text = """
data{
    int<lower=1> N;
    array[N] real y;
}
parameters{
    real mu;
    real<lower=0> sigma;
}
model{
    sigma ~ cauchy( 0 , 1 );
    mu ~ normal( 0 , 10 );
    y ~ normal( mu , sigma );
}
"""

In [ ]:
# Compile the model - note we do not need to save the text
model = stan.build(pystan_model1_text, data=stan_data, random_seed=123)

In [ ]:
# Fit the model
model1_pystan_fit = model.sample(num_chains=2,
                                 num_samples=4000,
                                 num_warmup=1000,
                                 save_warmup=True,
                                 init=[{'mu': 1, 'sigma': 2},
                                       {'mu': -1, 'sigma': 0.5}])

# Save the output to a file
with open("model1_pystan_fit.pkl", "wb") as f:
    pickle.dump(model1_pystan_fit, f)

In [ ]:
# Print the summary of the sampling results
model1_pystan_fit_az_idata = az.from_pystan(model1_pystan_fit, save_warmup=True)

model1_pystan_fit_az_idata

# Get summary statistics, including 2.75% and 97.5% quantiles
summary_stats = az.summary(model1_pystan_fit_az_idata,
                           var_names=["mu", "sigma"],
                           group="posterior",
                           hdi_prob=0.95,
                           kind="all"# 95% highest density interval
                           )

summary_stats

In [ ]:
# To plot traces and assess convergence, extract
# Monte Carlo samples including warmup in default
# array format that keeps all chains separate

# It's often helpful to plot the log posterior density too. Since it is a
# sample stat and arviz can only access one group at a time, we will manually
# add it to make it easier to view everything together
model1_pystan_fit_az_idata["posterior"]["lp"] = model1_pystan_fit_az_idata["sample_stats"]["lp"]
model1_pars = ["mu", "sigma"]
model1_pars_with_lp = model1_pars + ["lp"]

# Print summary of the parameters
summary_stats = az.summary(model1_pystan_fit_az_idata,
                           var_names=model1_pars_with_lp,
                           group="posterior",
                           hdi_prob=0.95,
                           kind="all"
                          )

summary_stats

In [ ]:
# Plot trace for parameters (mu, sigma, and lp)
# different colours are different chains
az.plot_trace(model1_pystan_fit_az_idata,
              filter_vars="like",
              var_names=model1_pars_with_lp,
              combined=False,
              compact=False,
              kind="trace",
              figsize=(11, 10))

plt.show()

# Save the plot to a PDF file
plt.savefig(output_dir.joinpath("model1_pystan_trace_arviz.pdf"),
            format="pdf",
            bbox_inches="tight")

plt.close()

In [ ]:
# Extract the posterior samples excluding warmup
# Make the pairs plot (including log-posterior)
az.plot_pair(model1_pystan_fit_az_idata,
             var_names=model1_pars_with_lp,
             marginals=True,
             figsize=(13, 8))

plt.show()

plt.savefig(output_dir.joinpath("model1_pystan_pairsplot.pdf"),
            format="pdf", bbox_inches="tight")
plt.close()